In [ ]:
import json
import requests
import matplotlib.pyplot as plt
import jsonpath_ng.ext as jp
from lxml import etree



 # Class 02 - Rest API y formatos de archivos



 ## Objetivos
 - Aprender a consumir una API REST
 - Aprender a leer y escribir archivos JSON
 - Aprender a leer y escribir archivos XML



 ## Formato de archivos json

In [ ]:


with open("data/example.json", "r", encoding="utf8") as f:
  # Cargamos el archivo JSON
  data = json.load(f)

# Los datos se cargan en un diccionario

In [ ]:


print("Header:", data["header"])

for i in data["esearchresult"]["idlist"]:
  print(f"Identificador : {i}")



 ## Otra opción: Jsonpath

In [ ]:


query = "$.esearchresult.idlist[*]"

jexpr = jp.parse(query)
# jexpr = jp.parse("$..idlist[*]")
# jexpr = jp.parse("$..idlist[:2]")
# jexpr = jp.parse("$..idlist[?(@ =~ '.*2$')]")

result = jexpr.find(data)

for i in result:
  print(f"Identificador : {i.value}")



 ## JSONPath
 Propósito: Consultar y extraer datos de estructuras JSON.

 - Elementos clave:
   - Sintaxis básica:
     - $: Raíz del documento.
     - . o []: Acceso a hijos (ej: $.genes[0].nombre).
     - *: Comodín para todos los elementos.
     - ..: Búsqueda recursiva (ej: $..secuencia).
   - Filtros:
     - ?(): Filtros condicionales (ej: $.genes[?(@.longitud > 1000)]).
     - Operadores lógicos: &&, ||, ==, !=.
   - Funciones:
     - Soporte limitado (varía por implementación): length(), keys().
   - Slice de arrays:
     - [start:end:step] (ej: $.genes[0:3]).



 ## Formato de archivos XML

In [ ]:


with open("data/example.xml", "r", encoding="utf8") as f:
  # Cargamos el archivo XML
  tree = etree.parse(f)

In [ ]:


# Query con XPath language

# tree.xpath("//IdList[Id]/Id/text()")
tree.xpath('//Id[position()<=2]/text()')
# tree.xpath(
#   '//Id[substring(text(), string-length(text()), 1) = "2"]/text()'
# )



 ## XPath

 - Propósito: Navegar y consultar documentos XML.
 - Elementos clave:
   - Sintaxis básica:
     - /: Raíz o separador de nodos (ej: /eSearchResult/IdList/Id).
     - //: Búsqueda en cualquier nivel (ej: //Hit_id).
     - @: Atributos (ej: //gen[@nombre="TP53"]).
   - Funciones:
     - text(): Obtener texto de un nodo.
     - contains(), starts-with(), substring().
     - count(), position(), last().
   - Ejes:
     - child::, parent::, descendant::, etc.
   - Operadores:
     - | (unión), =, !=, >, <.



 ## Consumiendo una API REST

In [ ]:


# Definimos la URL de la API

URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

# Definimos los parámetros de la consulta
params = {
  "db": "pubmed",
  "term": "cancer",
  "retmode": "json",
  "retmax": 10,
}

# Realizamos la consulta a la API

response = requests.get(URL, params=params, timeout=10)

# Verificamos el código de estado de la respuesta

if response.status_code == 200:
  # La consulta fue exitosa
  data = response.json()
  print("Consulta exitosa")
else:
  # La consulta falló
  print(f"Error en la consulta: {response.status_code}")

In [ ]:


identifiers = jp.parse("$.esearchresult.idlist[*]").find(data)
summaries = {}

for i in identifiers:
  URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
  params = {
    "db": "pubmed",
    "id": i.value,
    "retmode": "json",
  }
  response = requests.get(URL, params=params, timeout=10)
  if response.status_code == 200:
    data = response.json()
    summaries[i.value] = data
    print("Consulta exitosa")
  else:
    print(f"Error en la consulta: {i.value}/{response.status_code}")

In [ ]:


for i, summary in summaries.items():
  print(f"Title: {summary["result"][i]["title"]})")

In [ ]:


# Definir los parámetros de la consulta
latitude, longitude = -38.71761385288716, -62.26550770200703
payload = {
  "parameters": "T2M,PRECTOTCORR", # Temperatura y precipitación
  "community": "RE", # Tipo de usuario
  "longitude": longitude,
  "latitude": latitude,
  "start":  "20250301", # Fecha de inicio (YYYYMMDD)
  "end": "20250331", # Fecha de fin (YYYYMMDD)
  "format": "JSON", # Formato de respuesta
}

# Realizar la petición HTTP GET
response = requests.get(
  "https://power.larc.nasa.gov/api/temporal/daily/point",
  params = payload,
  timeout = 30
)

# Verificar el código de estado de la respuesta
if response.status_code == 200:
  data = response.json()
else:
  print(f"Error en la consulta: {response.status_code}")

In [ ]:


t2m, config_t2m = [
  x.value for x in jp.parse( "$..T2M[*]").find(data)
]
t2m = sorted(
  [
    (day, float(temp)) for day, temp in t2m.items()
    if temp > -900
  ]
)

In [ ]:


prectot, config_t2m = [
  x.value for x in jp.parse("$..PRECTOTCORR[*]").find(data)
]
prectot = sorted(
  [
    (day, float(temp)) for day, temp in prectot.items()
    if temp > -900
  ]
)

In [ ]:


fig, ax = plt.subplots(figsize=(10, 5))

days = [f"{day[4:6]}/{day[6:9]}" for day, _ in t2m]
ax.plot(
  range(len(t2m)),
  [temp for _, temp in t2m],
  label="Temperatura diaria",
  color="blue",
)
ax2 = ax.twinx()
ax2.plot(
  range(len(prectot)),
  [temp for _, temp in prectot],
  label="Precipitación diaria",
  color="green"
)
ax.set_xticks(range(len(t2m)))
ax.set_xticklabels(days, rotation=60, fontsize=10)
ax.set_xlabel("Fecha")
ax.set_ylabel("Temperatura (°C)")
ax2.set_ylabel("Precipitaciones (mm/day)")
ax.legend(loc="upper left")
ax2.legend(loc="upper right")

ax.set_title(
  "Temperatura y precipitaciones en el mes de marzo 2025, en Quilmes"
)
fig.tight_layout()

## Ejercicios



### 1. Recuperar datos de JSON

Leer el archivo 565671962.json y recuperar estos datos:

- número de acceso
- Número de version
- Organismo
- Taxonomy id
- Tipo de biomolécula
- Longitud de la molécula



### 2. Recuperar datos anidados de JSON

El archivo 40203473.json contiene datos de una publicación en Pubmed.
Leer el archivo y recuperar:

- Todos los autores.
- El último Autor.
- El DOI.
- La historia de publicación.


### 3. Recuperar datos de XML

El archivo 314295.xml tiene datos de una entrada de la base de datos taxonomy de
NCBI.

Leer el archivo y recuperar:

- El Rank
- DIvision
- Nombre científico
- Nombre Común
- TaxId

### 4. Recuperar datos XML anidados

El archivo 45534748.xml contiene una entrada de la base de datos BioSample.

Leer el archivo y recuperar todos los atributos, que no tengan el texto
"not aplicable".

### 5. Mapa de lluvias en una zona

Dados dos puntos en coordenadas geográficas (lat, long), generar una grilla
cuadrada de 5 x 5 puntos (donde los dos puntos originales son vértices
opuestos del cuadrado) y obtener las precipitaciones totales usando el
servicio de NASA Power (Ojo con la cantidad de consultas por minutos...)